In [1]:
import pandas as pd
import json
import csv
import numpy as np
import psycopg2
import ast
from sqlalchemy import create_engine, text

In [2]:
pd.set_option('display.max_columns', None)
compounds_df = pd.read_csv('homePage_data.csv')
compounds_df.head()

,id,compound_name,is_super,slug,properties_count,image_url,min_price,max_price,min_unit_area,max_unit_area,available_bathrooms,available_bedrooms,developer_id,developer_name,developer_logo,area_id,area_name,sponsored,max_installment_years,min_down_payment,min_ready_by,properties_ids,sum_10_properties_min_price,lat,long,has_offer,has_launches,is_launch,offer_title,limited_time_offer,property_types,property_types_count
0,195,Mountain View ICity New Cairo,True,195-mountain-view-icity-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,7584822,14559080,130,295,[3],[3],6,DMG,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,758483.0,2027-01-01,"[35486, 35487, 35489, 35490, 35495, 35497, 35502]",117896389.0,30.065182,31.593508,False,False,False,"5% Down Payment , instalments up to 9 years",True,"['Apartment', 'Villa', 'Townhouse', 'Duplex', ...","[{'id': 21, 'name': 'Apartment', 'count': 41, ..."
1,274,ZED,True,274-zed-el-sheikh-zayed-ora,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,10273202,20284992,160,301,"[3, 4]","[3, 4]",85,Ora Developers,https://s3.eu-central-1.amazonaws.com/prod.ima...,26,El Sheikh Zayed,0,8.0,513661.0,2024-10-31,"[41998, 41999, 42002, 42110, 42111, 42217, 42528]",129709414.0,30.046608,31.003101,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Penthouse', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 10, ..."
2,202,Bloomfields,True,202-bloomfields-tatweer-misr-mostakbal-city,76,https://s3.eu-central-1.amazonaws.com/prod.ima...,3664000,16210000,70,287,"[1, 2, 3, 4]","[1, 2, 3, 4]",33,Tatweer Misr,https://s3.eu-central-1.amazonaws.com/prod.ima...,10,Mostakbal City,0,8.0,209350.0,2023-12-30,"[17436, 31836, 31839, 31844, 31847, 31848, 318...",35147621.0,30.070208,31.649590,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 76, ..."
3,223,Badya,True,223-badya-palm-hills,42,https://s3.eu-central-1.amazonaws.com/prod.ima...,4782600,31084000,107,499,"[2, 3, 4, 6, 7, 8]","[2, 3, 4, 5, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,1,6th of October City,0,10.0,0.0,2023-11-29,"[9742, 11180, 13044, 16026, 19171, 19199, 1961...",44446502.0,29.854789,30.907116,False,False,False,NaN,True,"['Apartment', 'Villa', 'Townhouse', 'Penthouse...","[{'id': 21, 'name': 'Apartment', 'count': 48, ..."
4,148,Palm Hills New Cairo,True,148-palm-hills-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,18886000,72968000,190,611,"[4, 7, 8]","[3, 4, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,944300.0,2022-12-30,"[22676, 23315, 28769, 28770, 32932, 37716, 42215]",487964000.0,30.059658,31.598640,False,False,False,NaN,True,"['Apartment', 'Villa', 'Twinhouse', 'Townhouse']","[{'id': 21, 'name': 'Apartment', 'count': 13, ..."


In [3]:
# insert data in table compound
# Frist handle data to be like we need to insert
compound_table = compounds_df

def _max_or_min(lst, is_max=True):
    try:
        _list = ast.literal_eval(lst)
        if is_max:
            return max(_list) if _list else None
        else:
            return min(_list) if _list else None
    except Exception as e:
        return None

# Apply _max_or_min function for max values
compounds_df['max_available_bathrooms'] = compounds_df['available_bathrooms'].apply(_max_or_min, is_max=True)
compound_table['max_available_bedrooms'] = compounds_df['available_bedrooms'].apply(_max_or_min, is_max=True)

# Apply _max_or_min function for min values
compounds_df['min_available_bathrooms'] = compounds_df['available_bathrooms'].apply(_max_or_min, is_max=False)
compound_table['min_available_bedrooms'] = compounds_df['available_bedrooms'].apply(_max_or_min, is_max=False)


In [5]:
#select attributes to load
compound_table = compound_table[['id', 'compound_name', 'is_super',
                                'properties_count', 'area_id', 'developer_id',
                                'min_price', 'max_price', 'min_unit_area', 'max_unit_area',
                                'max_available_bathrooms', 'min_available_bathrooms',
                                'max_available_bedrooms', 'min_available_bedrooms',
                                'sponsored', 'max_installment_years','min_down_payment',
                                'sum_10_properties_min_price', 'has_offer',
                                'offer_title', 'limited_time_offer', 'has_launches',
                                'is_launch', 'min_ready_by']]

In [6]:
# connnect to db 
# Replace these with your database details
dbname = "nawy"
user = "nady"
password = "admin"
host = "172.20.0.3"
port = "5432"


# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

except psycopg2.Error as e:
    print("Error connecting to the database:", e)
    exit()


# Create SQLalchemy engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")


In [7]:
# maping attribute to be like db
column_mapping = {
    'id': 'compound_id',
    'compound_name': 'compound_name',
    'area_id': 'area_id',
    'developer_id':'developer_id',
    'has_launches':'has_launches',
    'has_offer':'has_offer',
    'is_launch':'is_launch',
    'is_super':'is_super',
    'limited_time_offer':'limited_time_offer',
    'max_available_bathrooms': 'max_available_bathrooms',
    'max_available_bedrooms':'max_available_bedrooms',
    'max_installment_years':'max_installment_years',
    'max_price':'max_price',
    'max_unit_area':'max_unit_area',
    'min_available_bathrooms' :'min_available_bathrooms',
    'min_available_bedrooms':'min_available_bedrooms',
    'min_down_payment':'min_down_payment',
    'min_price':'min_price',
    'min_ready_by':'min_ready_by',
    'min_unit_area':'min_unit_area',
    'offer_title':'offer_title',
    'properties_count':'properties_count',
    'sponsored':'sponsored',
    'sum_10_properties_min_price':'sum_10_properties_min_price'
    }
compound_table.rename(columns=column_mapping, inplace=True)
columns = sorted(compound_table.columns) 

In [8]:
#check df has the same db tables column name
#cursor = connection.cursor()

# Query to retrieve column names
with engine.connect() as connection:
    query = text("SELECT column_name FROM information_schema.columns WHERE table_name = 'compound' ORDER BY column_name ASC;")
    results = connection.execute(query)

    # Fetch the results
    column_names = results.fetchall()

# Display the column names
for row, column_name in enumerate(column_names):
    results = column_name[0] == columns[row]
    if results == True:
        pass
    else:
        print(f"{column_name[0]} vs {columns[row]}")

In [9]:
## match column datatypes in db table with column datatypes in df table
column_types = compound_table.dtypes.sort_index()
column_types
with engine.connect() as connection:
    query = text(
        f"select column_name, data_type from information_schema.columns where table_name='compound' ORDER BY column_name"
    )
    # Excute the query
    results = connection.execute(query)

    #fetch the result
    column_info = results.fetchall()
for row, column_ in enumerate(column_info):
    results = column_[0] == columns[row]
    if results == True:
        print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")
    else:
        print(f"{column_name[0]} vs {columns[row]}")

area_id                        --> integer             , :: int64
compound_id                    --> integer             , :: int64
compound_name                  --> character varying   , :: object
developer_id                   --> integer             , :: int64
has_launches                   --> boolean             , :: bool
has_offer                      --> boolean             , :: bool
is_launch                      --> boolean             , :: bool
is_super                       --> boolean             , :: object
limited_time_offer             --> boolean             , :: bool
max_available_bathrooms        --> integer             , :: float64
max_available_bedrooms         --> integer             , :: float64
max_installment_years          --> integer             , :: float64
max_price                      --> numeric             , :: int64
max_unit_area                  --> integer             , :: int64
min_available_bathrooms        --> integer             , :: float64
min_

/tmp/ipykernel_74088/3784703426.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")


In [10]:
## convert non mathcing data types for inserting
compound_table.loc[:, 'is_super'] = compound_table.loc[:,'is_super'].astype(bool)
compound_table.loc[:, 'sponsored'] = compound_table.loc[:, 'sponsored'].astype(bool)
compound_table.loc[:, 'min_ready_by'] = pd.to_datetime(compound_table.loc[:, 'min_ready_by'])
compound_table.loc[:, 'max_available_bathrooms'] = compound_table.loc[:, 'max_available_bathrooms'].fillna(0).astype(int)
compound_table.loc[:, 'max_available_bedrooms'] = compound_table.loc[:, 'max_available_bedrooms'].fillna(0).astype(int)
compound_table.loc[:, 'max_installment_years'] = compound_table.loc[:, 'max_installment_years'].fillna(0).astype(int)
compound_table.loc[:, 'min_available_bathrooms'] = compound_table.loc[:, 'min_available_bathrooms'].fillna(0).astype(int)
compound_table.loc[:, 'min_available_bedrooms'] = compound_table.loc[:, 'min_available_bedrooms'].fillna(0).astype(int)
compound_table.loc[:, 'min_down_payment'] = compound_table.loc[:, 'min_down_payment'].fillna(0).astype(int)
compound_table.loc[:, 'min_unit_area'] = compound_table.loc[:, 'min_unit_area'].fillna(0).astype(int)
compound_table.loc[:, 'max_price'] = compound_table.loc[:, 'max_price'].astype(float)
compound_table.loc[:, 'min_price'] = compound_table.loc[:, 'min_price'].astype(float)
compound_table.loc[:, 'sum_10_properties_min_price'] = compound_table.loc[:, 'sum_10_properties_min_price'].astype(float)


In [11]:
# insert data to database
compound_table.to_sql('compound', engine, if_exists='append', index=False)

530